imports

In [1]:
import os
import pandas as pd
from HGNN.train import CSV_processor

parameters

In [2]:
dataPath="/raid/elhamod/Fish/Curated3/"
image_path='Easy'
output_directory_name="curated_30_50" #curated_minimum5images"#"biology_paper_cleaned_30max"
output_cleaned_species_csv_fileName = "cleaned_metadata.csv"
input_species_csv_fileName = "cleaned_metadata.csv"#"biology_paper_cleaned_all/cleaned_metadata.csv"
numberOfImagesPerSpecies_min =30 # If None, no min
numberOfImagesPerSpecies_toKeep =50 # If None, keep all

cleanup metadata

In [3]:
# csv_processor = CSV_processor.CSV_processor(os.path.join(dataPath, image_path), '', cleanup=True)

Subset creation

In [4]:
# get metadata file
metadata_csv_df = pd.read_csv(os.path.join(dataPath ,image_path, input_species_csv_fileName), sep='\t')

# find species that have a specific number of images
grouped_by = metadata_csv_df['scientificName'].value_counts()
if numberOfImagesPerSpecies_min is not None:
    grouped_by = grouped_by[grouped_by > numberOfImagesPerSpecies_min]
scientificNameValues = grouped_by.index.values
thresholded = metadata_csv_df[metadata_csv_df['scientificName'].isin(scientificNameValues)]

# Get first n images from each
if numberOfImagesPerSpecies_toKeep is not None:
    trimmed = thresholded.groupby('scientificName', group_keys=False)
    trimmed = trimmed.apply(lambda x: x.sample(n=numberOfImagesPerSpecies_toKeep) if len(x) >= numberOfImagesPerSpecies_toKeep else x)
#     trimmed = trimmed.head(numberOfImagesPerSpecies_toKeep)
#     trimmed = trimmed.sample(10)
    trimmed.reset_index(drop=True)
else:
    trimmed = thresholded
print('output', trimmed)

# save the file
path = os.path.join(dataPath ,image_path, output_directory_name)
try:
    os.mkdir(path)
except:
    pass
trimmed.to_csv(os.path.join(path, output_cleaned_species_csv_fileName), sep='\t')


output                   fileName         scientificName        Genus      Family
13     INHS_FISH_48349.JPG    Alosa chrysochloris        Alosa   Clupeidae
21    INHS_FISH_106186.JPG    Alosa chrysochloris        Alosa   Clupeidae
44      INHS_FISH_5773.JPG    Alosa chrysochloris        Alosa   Clupeidae
58     INHS_FISH_25323.JPG    Alosa chrysochloris        Alosa   Clupeidae
23    INHS_FISH_105918.JPG    Alosa chrysochloris        Alosa   Clupeidae
...                    ...                    ...          ...         ...
6372   INHS_FISH_19332.jpg  Phenacobius mirabilis  Phenacobius  Cyprinidae
6405   INHS_FISH_91842.jpg  Phenacobius mirabilis  Phenacobius  Cyprinidae
6302   INHS_FISH_25634.jpg  Phenacobius mirabilis  Phenacobius  Cyprinidae
6442   INHS_FISH_57438.jpg  Phenacobius mirabilis  Phenacobius  Cyprinidae
6386   INHS_FISH_16284.jpg  Phenacobius mirabilis  Phenacobius  Cyprinidae

[1900 rows x 4 columns]
